# EDA for Amazon Stars vs Sentiment

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

## Load Parquet

In [ ]:
df = pd.read_parquet('data/raw.parquet')  # placeholder
df.head()

## Star-rating distribution

In [ ]:
sns.countplot(x='star_rating', data=df);
plt.savefig('results/star_counts.png')

## Sentiment scoring

In [ ]:
from transformers import pipeline
sent_clf = pipeline('sentiment-analysis')
df['sent_score'] = df['review_body'].apply(lambda x: sent_clf(x)[0]['score'])

## Divergence metric

In [ ]:
df['divergence'] = df['sent_score'] - (df['star_rating']-3)/2
ax = df['divergence'].hist()
plt.savefig('results/divergence_hist.png')

## Polarity vs Rating

In [ ]:
sns.boxplot(x='star_rating', y='sent_score', data=df);
plt.savefig('results/polarity_vs_rating.png')

## Helpful votes vs Divergence

In [ ]:
sns.scatterplot(x='helpful_votes', y='divergence', data=df);
plt.savefig('results/helpful_vs_div.png')

## Save cleaned Parquet

In [ ]:
df.to_parquet('data/clean_1M.parquet')